In [ ]:
using DifferentialEquations
using Plots
using SkeelBerzins

In [ ]:
N_x = 21

L = 1
T = 10

x_mesh = collect(range(0, L; length=N_x))
tspan = (0, T)

m = 0

function pdefun(x, t, u, dudx)
    c = SVector(1, 1)
    c = SVector(1, 0) # Mod TD
    f = SVector(0.5, 0.1) .* dudx
    y = u[1] - u[2]
    s = SVector(-y, y)

    return c, f, s
end

function icfun(x)
    u0 = SVector(0.0, 0.0)

    return u0
end

function bdfun(xl, ul, xr, ur, t)

    # Dirichlet-1 for first, Neumann-0 for second species
    pl = SVector(ul[1] - 1.0, 0)
    ql = SVector(0, 1)

    # Neumann-0 for first, Dirichlet-0 for second species
    pr = SVector(0, ur[2])
    pr = SVector(-1, ur[2]) # mod TD
    qr = SVector(1, 0)

    return pl, ql, pr, qr
end

In [ ]:
params_diffEq = SkeelBerzins.Params(; solver=:DiffEq)

pb = pdepe(m, pdefun, icfun, bdfun, x_mesh, tspan; params=params_diffEq)
problem = DifferentialEquations.ODEProblem(pb)
sol_diffEq = DifferentialEquations.solve(problem, Rosenbrock23())
sol_reshaped_diffEq = reshape(sol_diffEq, pb)

In [ ]:
c1 = sol_reshaped_diffEq.u[end][1, :]
c2 = sol_reshaped_diffEq.u[end][2, :]

plot(x_mesh, c1)
plot!(x_mesh, c2)